In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
dfE = pd.read_csv("../input/dfe-imported/dfE.csv")
dfE

In [ ]:
# calculate plot's boundary
x_center = (dfE['X'].max() + dfE['X'].min())/2
y_center = (dfE['Y'].max() + dfE['Y'].min())/2
x_half_range = (dfE['X'].max() - dfE['X'].min())/2
y_half_range = (dfE['Y'].max() - dfE['Y'].min())/2
x_range = (x_center - x_half_range, x_center + x_half_range)
y_range = (y_center - y_half_range, y_center + y_half_range)

sw = (x_range[0], y_range[0])
ne = (x_range[1], y_range[1])
boundary = zip(sw,ne)  # Create zip object with ranges

plot_width = int(1024)
# calculate plot's height from its width
plot_height = int(plot_width/(x_half_range/y_half_range))

In [ ]:
import datashader as ds
from functools import partial
from datashader.utils import export_image
from datashader import transfer_functions as tf
from datashader.colors import colormap_select, Greys9, Hot, viridis, inferno
from IPython.core.display import HTML, display

In [ ]:
background = 'black'

# We set up partial objects “export” and “cm”:
    # “export” helps to export the plot created to a predetermined path, while 
    #  “cm” helps with the color mapping.
export = partial(export_image, background=background, export_path='export_SDH')
cm = partial(colormap_select, reverse=(background!='black'))

display(HTML('<style>.container {width:100% !important; }</style>'))

In [ ]:
cvs = ds.Canvas(plot_width, plot_height, *boundary)
agg = cvs.points(dfE, 'X', 'Y')

# The cvs object ds.Canvas() is set up with predetermined widths and heights. The boundary object containing the bounds of the map are also passed in.
# Points are generated on the canvas using the X, Y pair passed in.

In [ ]:
# Now that the setup is complete, we can start plotting and exporting the plots with a single line of code as shown below. 
# Color maps, normalizations are all parameters that can be readily changed.
export(tf.shade(agg, cmap=cm(Greys9, 0.2), how='eq_hist'), "TorontoCollisions_grey")

In [ ]:
from colorcet import fire
export(tf.shade(agg, cmap=cm(fire, 0.2), how='log'), "TorontoCollisions_fire")

In [ ]:
export(tf.shade(agg, cmap=cm(viridis, 0.2), how='eq_hist'), "TorontoCollisions_viridis")

In [ ]:
# Enough with the black background though. How would you overlay these points on an actual map?

import holoviews as hv
from holoviews.element.tiles import EsriImagery
from holoviews.operation.datashader import datashade
hv.extension('bokeh')

map_tiles  = EsriImagery().opts(alpha=0.5, width=900, height=480, bgcolor='black')
points     = hv.Points(dfE, ['X', 'Y'])
accidents  = datashade(points, x_sampling=10, y_sampling=10, cmap=fire, width=900, height=480)

map_tiles * accidents